<a href="https://colab.research.google.com/github/yj9889/ESAA2/blob/main/220328%EC%95%99%EC%83%81%EB%B8%94%EC%97%B0%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1-a. MNIST 데이터를 훈련, 검증, 테스트 데이터로 나눈다.

In [14]:
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784',version=1)

In [15]:
from sklearn.model_selection import train_test_split

X_train_val, X_test, y_train_val, y_test = train_test_split(mnist.data,mnist.target, test_size=10000, random_state=42)
X_train,X_val,y_train,y_val = train_test_split(X_train_val,y_train_val,test_size=10000, random_state=42)

1-b. 랜덤 포레스트 분류기, 엑스트라 트리 분류기, SVM 분류기, MLP 분류기를 훈련시킨다. (n_estimators=100, random_state=42)

In [16]:
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
random_forest_clf = RandomForestClassifier(n_estimators=100, random_state=42)
extra_trees_clf = ExtraTreesClassifier(n_estimators=100, random_state=42)
svm_clf = LinearSVC(random_state=42)
mlp_clf = MLPClassifier(random_state=42)

In [17]:
estimators = [random_forest_clf, extra_trees_clf, svm_clf, mlp_clf]
for estimator in estimators:
    print("Training the", estimator)
    estimator.fit(X_train, y_train)

Training the RandomForestClassifier(random_state=42)
Training the ExtraTreesClassifier(random_state=42)
Training the LinearSVC(random_state=42)


/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,


Training the MLPClassifier(random_state=42)


In [18]:
[estimator.score(X_val, y_val) for estimator in estimators]

[0.9692, 0.9715, 0.8695, 0.9639]

1-c. 이들을 직접 투표 분류기를 사용하는 앙상블로 연결한다.

In [19]:
from sklearn.ensemble import VotingClassifier
named_estimators = [
    ("random_forest_clf", random_forest_clf),
    ("extra_trees_clf", extra_trees_clf),
    ("svm_clf", svm_clf),
    ("mlp_clf", mlp_clf),
]

In [20]:
voting_clf = VotingClassifier(named_estimators)
voting_clf.fit(X_train, y_train)

voting_clf.score(X_val, y_val)
[estimator.score(X_val, y_val) for estimator in voting_clf.estimators_]

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,


[0.0, 0.0, 0.0, 0.0]

1-d. 다른 분류기보다 성능이 많이 떨어졌던 선형 SVM 제거 후 VotingClassifier를 다시 평가

In [21]:
voting_clf.set_params(svm_clf=None)
voting_clf.estimators_

[('random_forest_clf', RandomForestClassifier(random_state=42)),
 ('extra_trees_clf', ExtraTreesClassifier(random_state=42)),
 ('svm_clf', None),
 ('mlp_clf', MLPClassifier(random_state=42))]

In [22]:
del voting_clf.estimators_[2] # 안지워져서 다시

voting_clf.score(X_val, y_val)

0.9736

1-d. 간접 투표 분류기를 사용하여 앙상블로 연결.

In [23]:
# 이미 훈련 시켰으니 Hard -> soft 바꾸기
voting_clf.voting = "soft"
voting_clf.score(X_val, y_val)

0.97

In [24]:
# 개개의 분류기보다 낫다
voting_clf.score(X_test,y_test)

0.9671

In [25]:
[model.score(X_test,y_test) for model in voting_clf.estimators_]

[0.0, 0.0, 0.0]